In [1]:
import pandas as pd
df = pd.read_feather("Data.ftr")
import time
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

## Dropping the customer id 

In [2]:
df

,index,P_2,year,month,day,D_39,B_1,B_2,R_1,S_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,Target
0,1432714,0.674805,2017.0,6.0,22.0,0.596680,0.031982,1.007812,0.008217,0.174927,...,NaN,NaN,0.001536,0.004150,0.002407,NaN,0.009689,0.001431,0.009621,0
1,4278638,0.694336,2018.0,1.0,4.0,0.268311,0.055847,0.817871,0.006561,0.174072,...,NaN,NaN,1.003906,0.005562,0.929199,0.337402,1.001953,0.003347,0.093628,0
2,1821472,0.850586,2018.0,1.0,4.0,0.004910,0.005058,0.811523,0.008026,0.192871,...,NaN,NaN,0.000739,0.007881,0.000373,NaN,0.000974,0.005032,0.006668,0
3,3826135,0.815918,2017.0,5.0,12.0,0.005737,0.010452,1.007812,0.002213,0.117981,...,NaN,NaN,0.005707,0.007473,0.007141,NaN,0.008278,0.001192,0.007935,0
4,5253960,0.899902,2018.0,3.0,26.0,0.090088,0.010933,1.008789,0.008041,0.122437,...,NaN,NaN,0.008476,0.007244,0.005856,NaN,0.006340,0.008934,0.003929,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212575,5030793,0.163818,2017.0,7.0,6.0,0.122192,0.162476,0.135132,0.001577,0.146484,...,NaN,NaN,0.004353,0.006596,0.008560,NaN,0.007706,0.009628,0.003382,1
2212576,491263,0.402588,2017.0,6.0,3.0,0.038605,0.072083,0.811523,0.004528,0.177856,...,NaN,NaN,0.004704,0.009277,0.001515,NaN,0.003225,0.005203,0.007736,1
2212577,3937352,0.812500,2017.0,8.0,24.0,0.210083,0.148315,0.041138,0.009972,0.096680,...,NaN,NaN,0.002039,0.003349,0.007843,NaN,0.002094,0.004456,0.001372,0
2212578,4686059,0.700195,2017.0,9.0,11.0,0.007904,0.021210,1.003906,0.023834,0.123962,...,NaN,NaN,0.005547,0.008049,0.007561,NaN,0.006004,0.005024,0.004902,0


In [3]:
df.drop('index',axis=1,inplace=True)

In [4]:
X, y = df.drop("Target", axis = 1, inplace = False), df["Target"]

## One hot encoding 

In [5]:
X_cat_mean =pd.get_dummies(X, columns=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'], drop_first=True)
X_cat_Normalize = pd.get_dummies(X, columns=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'], drop_first=True)

# Filling Null Values 

In [6]:
# Fill missing values with mean column values in the train set
for col in X_cat_mean.columns:
    X_cat_mean[col].fillna((X_cat_mean[col].mean()), inplace=True)

X_cat_mean.fillna(X_cat_mean['D_142'].astype(float).mean(),inplace=True)
X_cat_mean.fillna(X_cat_mean['P_2'].astype(float).mean(),inplace=True)


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cat_mean, y, test_size=0.3)

In [8]:
seconds1 = time.time()
from xgboost import XGBClassifier as xgb
xgb_clf = xgb()
xgb_clf.fit(X_train, y_train)
results1=xgb_clf.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,results1)
seconds2= seconds = time.time()
time1=seconds1-seconds2
print(time1)

-441.55473828315735


In [10]:
# Calculate precision and recall
from sklearn.metrics import precision_score, recall_score
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results1))


precision
0.7628668535261107
recall
0.7579176910484636
accuracy
0.8808660779120604
F1
0.7603842191382342


0.7582896187441697

In [11]:
# Perform a comparitive analysis of the models
from sklearn.metrics import classification_report
print(classification_report(y_test,results1))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92    498688
           1       0.76      0.76      0.76    165086

    accuracy                           0.88    663774
   macro avg       0.84      0.84      0.84    663774
weighted avg       0.88      0.88      0.88    663774



0.8518803086592726

# Mode 

In [11]:
# Perform One Hot Encoding on categorical features
X_cat_mode = pd.get_dummies(X, columns=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'], drop_first=True)

In [12]:
# Fill missing values with mode column values in the train set
for col in X_cat_mode.columns:
    X_cat_mode[col].fillna((X_cat_mode[col].mode()[0]), inplace=True)

X_cat_mode.fillna(X_cat_mode['D_142'].astype(float).mode()[0],inplace=True)
X_cat_mode.fillna(X_cat_mode['P_2'].astype(float).mode()[0],inplace=True)

In [13]:
# Split the data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X_cat_mode, y, test_size=0.3)

# Train the model
seconds1 = time.time()
xgb_clf.fit(X_train, y_train)
seconds2= seconds = time.time()
print(seconds2-seconds1)
results2=xgb_clf.predict(X_test)

# Calculate accuracy score


436.9243462085724


In [15]:
# Calculate precision and recall
from sklearn.metrics import precision_score, recall_score
print("precision")
print(precision_score(y_test, results2))
print("recall")
print(recall_score(y_test, results2))
print("accuracy")
print(accuracy_score(y_test,results2))
print("F1")
print(f1_score(y_test,results2))

precision
0.7635547000320821
recall
0.7613058199397666
accuracy
0.8815711371641552
F1
0.7624286016500952


# Median

In [16]:
# Perform One Hot Encoding on categorical features
X_cat_median = pd.get_dummies(X, columns=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'], drop_first=True)

# Fill missing values with median column values in the train set
for col in X_cat_median.columns:
    X_cat_median[col].fillna((X_cat_median[col].median()), inplace=True)

X_cat_median.fillna(X_cat_median['D_142'].astype(float).median(),inplace=True)
X_cat_median.fillna(X_cat_median['P_2'].astype(float).median(),inplace=True)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_cat_median, y, test_size=0.3)

In [17]:
# Train the model
seconds1 = time.time()
xgb_clf = xgb()
xgb_clf.fit(X_train, y_train)
seconds2=time.time()
print(seconds2-seconds1)
results3=xgb_clf.predict(X_test)



434.74010586738586


In [19]:
# Calculate precision and recall
print("precision")
print(precision_score(y_test, results3))
print("recall")
print(recall_score(y_test, results3))
print("accuracy")
print(accuracy_score(y_test,results3))
print("F1")
print(f1_score(y_test,results3))

precision
0.7631732133545156
recall
0.7591683354645692
accuracy
0.8810076923772248
F1
0.7611655065224096


## training without any preprocessing  

In [20]:
# Perform One Hot Encoding on categorical features
X_cat_zero = pd.get_dummies(X, columns=['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'], drop_first=True)

# Directly use the data without any preprocessing
X_train, X_test, y_train, y_test = train_test_split(X_cat_zero, y, test_size=0.3)

# Train the model
sleep1=time.time()
xgb_clf = xgb()
xgb_clf.fit(X_train, y_train)
sleep2=time.time()
print(sleep2-sleep1)
results4=xgb_clf.predict(X_test)


513.3358521461487


In [21]:
print("precision")
print(precision_score(y_test, results4))
print("recall")
print(recall_score(y_test, results4))
print("accuracy")
print(accuracy_score(y_test,results4))
print("F1")
print(f1_score(y_test,results4))

precision
0.7627207182035194
recall
0.7600154945859737
accuracy
0.8814144573303564
F1
0.7613657034063761


In [22]:
X_cat_zero['D_137']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
2212575   NaN
2212576   NaN
2212577   NaN
2212578   NaN
2212579   NaN
Name: D_137, Length: 2212580, dtype: float16

## Normilization 

In [23]:
for col in X_cat_mean.columns:
    X_cat_Normalize[col].fillna((X_cat_mean[col].mean()), inplace=True)

X_cat_Normalize.fillna(X_cat_mean['D_142'].astype(float).mean(),inplace=True)
X_cat_Normalize.fillna(X_cat_mean['P_2'].astype(float).mean(),inplace=True)

In [24]:

from sklearn import preprocessing
import pandas as pd
d = preprocessing.normalize(X_cat_Normalize)
names = X_cat_Normalize.columns
scaled_df = pd.DataFrame(d, columns=names)
scaled_df

,P_2,year,month,day,D_39,B_1,B_2,R_1,S_3,D_41,...,D_64_O,D_64_R,D_64_U,D_66_1.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0
0,0.000335,0.999932,0.002975,0.010907,0.000296,0.000016,0.000500,4.073525e-06,0.000087,5.011533e-07,...,0.000496,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000496
1,0.000344,0.999994,0.000496,0.001982,0.000133,0.000028,0.000405,3.251359e-06,0.000086,8.681318e-07,...,0.000000,0.000496,0.000000,0.000496,0.0,0.000000,0.000496,0.000000,0.000000,0.000000
2,0.000421,0.999994,0.000496,0.001982,0.000002,0.000003,0.000402,3.977243e-06,0.000096,3.026409e-06,...,0.000496,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000496
3,0.000405,0.999975,0.002479,0.005949,0.000003,0.000005,0.000500,1.096911e-06,0.000058,4.315777e-06,...,0.000496,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000496
4,0.000446,0.999912,0.001486,0.012883,0.000045,0.000005,0.000500,3.984478e-06,0.000061,1.777705e-06,...,0.000495,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2212575,0.000081,0.999986,0.003470,0.002975,0.000061,0.000081,0.000067,7.820305e-07,0.000073,2.091719e-06,...,0.000000,0.000496,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000496,0.000000
2212576,0.000200,0.999992,0.002975,0.001487,0.000019,0.000036,0.000402,2.244922e-06,0.000088,3.058162e-06,...,0.000000,0.000000,0.000496,0.000000,0.0,0.000496,0.000000,0.000000,0.000000,0.000000
2212577,0.000403,0.999918,0.003966,0.011898,0.000104,0.000074,0.000020,4.943382e-06,0.000048,4.932035e-06,...,0.000000,0.000496,0.000000,0.000496,0.0,0.000000,0.000000,0.000496,0.000000,0.000000
2212578,0.000347,0.999972,0.004462,0.005453,0.000004,0.000011,0.000498,1.181634e-05,0.000061,1.448674e-06,...,0.000000,0.000000,0.000496,0.000000,0.0,0.000000,0.000000,0.000496,0.000000,0.000000


In [25]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_df, y, test_size=0.3)
from xgboost import XGBClassifier as xgb
xgb_clf = xgb()
sleep1=time.time()
xgb_clf.fit(X_train, y_train)
print(time.time()-sleep1)
results1=xgb_clf.predict(X_test)



736.8607220649719


In [26]:
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results1))

precision
0.761237877750789
recall
0.7593749622342804
accuracy
0.8806235254770449
F1
0.7603052788558395


In [27]:
from sklearn.decomposition import PCA
tranformer_PCA=PCA(0.99)
tranformer_PCA.fit(scaled_df)
X_cat_pca= tranformer_PCA.transform(scaled_df)

In [28]:
map={}
count = 0
for i in X_cat_pca:
    obj={count:i}
    map.update(obj)
    
len(map[0])


66

In [29]:
columns=[]
for i in range(len(map[0])):
    col = 'PCA'+str(i)
    columns.append(col)

In [30]:
X_pca=pd.DataFrame(X_cat_pca,columns=columns)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3)
from xgboost import XGBClassifier as xgb
sleep1=time.time()
xgb_clf = xgb()
xgb_clf.fit(X_train, y_train)
print(time.time()-sleep1)
results1=xgb_clf.predict(X_test)


266.52031993865967


In [33]:
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results1))

precision
0.745803772278941
recall
0.7295545775499347
accuracy
0.8706366926092315
F1
0.7375896928135736


## Lable encoding 

In [34]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.


arr = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

for i in arr:
    df[i]= label_encoder.fit_transform(df[i])
X_cat_label=df

In [35]:
X_cat_label, y = df.drop("Target", axis = 1, inplace = False), df["Target"]

In [36]:

for col in X_cat_label.columns:
    X_cat_label[col].fillna((X_cat_label[col].mean()), inplace=True)

X_cat_label.fillna(X_cat_label['D_142'].astype(float).mean(),inplace=True)
X_cat_label.fillna(X_cat_label['P_2'].astype(float).mean(),inplace=True)

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_cat_label, y, test_size=0.3)

In [38]:
from xgboost import XGBClassifier as xgb
xgb_clf = xgb()
sleep1=time.time()
xgb_clf.fit(X_train, y_train)
print(time.time()-sleep1)
results1=xgb_clf.predict(X_test)



351.8077688217163


In [39]:
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results1))

precision
0.761800792076806
recall
0.7596216647717652
accuracy
0.8809489374395502
F1
0.7607096678466201


## SVD 

In [40]:
from sklearn.decomposition import TruncatedSVD 


In [41]:
truncatedSVD=TruncatedSVD(30)

In [42]:
X_truncated = truncatedSVD.fit_transform(X_cat_mean)

In [43]:
columns=[]
for i in range(30):
    col = 'SVD'+str(i)
    columns.append(col)

In [44]:
X_svd=pd.DataFrame(X_truncated,columns=columns)

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_svd, y, test_size=0.3)

In [46]:
from xgboost import XGBClassifier as xgb
xgb_clf = xgb()
sleep1=time.time()
xgb_clf.fit(X_train, y_train)
print(time.time()-sleep1)
results1=xgb_clf.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test,results1)

130.08787608146667


0.8672545173507851

In [48]:
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results1))


precision
0.7377518076134592
recall
0.7258957743246589
accuracy
0.8672545173507851
F1
0.7317757720582639


## SVD 

In [49]:
from sklearn.decomposition import TruncatedSVD 
truncatedSVD=TruncatedSVD(70)
X_truncated = truncatedSVD.fit_transform(X_cat_mean)
columns=[]
for i in range(70):
    col = 'SVD'+str(i)
    columns.append(col)
X_svd=pd.DataFrame(X_truncated,columns=columns)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_svd, y, test_size=0.3)
from xgboost import XGBClassifier as xgb
xgb_clf = xgb()
sleep1=time.time()
xgb_clf.fit(X_train, y_train)
print(time.time()-sleep1)
results1=xgb_clf.predict(X_test)
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results3))

269.8506512641907
precision
0.745282379541268
recall
0.7302029951288412
accuracy
0.8703745551949911
F1
0.24984648519149272


In [50]:
from sklearn.decomposition import TruncatedSVD 
truncatedSVD=TruncatedSVD(100)
X_truncated = truncatedSVD.fit_transform(X_cat_mean)
columns=[]
for i in range(100):
    col = 'SVD'+str(i)
    columns.append(col)
X_svd=pd.DataFrame(X_truncated,columns=columns)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_svd, y, test_size=0.3)
from xgboost import XGBClassifier as xgb
xgb_clf = xgb()
sleep1=time.time()
xgb_clf.fit(X_train, y_train)
print(time.time()-sleep1)
results1=xgb_clf.predict(X_test)
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results3))

502.5852520465851
precision
0.7474302132613381
recall
0.7339770875392212
accuracy
0.8726433997113475
F1
0.2476060357652488


In [51]:
from sklearn.decomposition import TruncatedSVD 
truncatedSVD=TruncatedSVD(120)
X_truncated = truncatedSVD.fit_transform(X_cat_mean)
columns=[]
for i in range(120):
    col = 'SVD'+str(i)
    columns.append(col)
X_svd=pd.DataFrame(X_truncated,columns=columns)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_svd, y, test_size=0.3)
from xgboost import XGBClassifier as xgb
xgb_clf = xgb()
sleep1=time.time()
xgb_clf.fit(X_train, y_train)
print(time.time()-sleep1)
results1=xgb_clf.predict(X_test)
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results3))

451.9024670124054
precision
0.7505637135285069
recall
0.7350341514764691
accuracy
0.8731887660559167
F1
0.24814436829784592


In [52]:
from sklearn.decomposition import TruncatedSVD 
truncatedSVD=TruncatedSVD(140)
X_truncated = truncatedSVD.fit_transform(X_cat_mean)
columns=[]
for i in range(140):
    col = 'SVD'+str(i)
    columns.append(col)
X_svd=pd.DataFrame(X_truncated,columns=columns)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_svd, y, test_size=0.3)
from xgboost import XGBClassifier as xgb
xgb_clf = xgb()
sleep1=time.time()
xgb_clf.fit(X_train, y_train)
print(time.time()-sleep1)
results1=xgb_clf.predict(X_test)
print("precision")
print(precision_score(y_test, results1))
print("recall")
print(recall_score(y_test, results1))
print("accuracy")
print(accuracy_score(y_test,results1))
print("F1")
print(f1_score(y_test,results3))

452.78914403915405
precision
0.7514213069218729
recall
0.7369152272961675
accuracy
0.8741348712061635
F1
0.2480143867874461
